# Results Testing

Notebook for creating results tables and plots and testing them interactively. Note that the plots are also made by ``compute_results.py``.

### Set up

In [ ]:
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nestcheck.ns_run_utils
import bsr.data
import bsr.likelihoods
import bsr.basis_functions as bf
import bsr.neural_networks as nn
import bsr.plotting
import bsr.results_utils
import bsr.results_tables
import bsr.paper_plots
%matplotlib inline

### Settings

Specify the problem. See ``compute_results.py`` for an explanation of the settings.

In [ ]:
# Data and function to fit it with
# --------------------------------
fit_func_list = [bf.gg_1d]
data_func_list = [bf.gg_1d]
data_type_list = [3]
inds = list(range(1, 6))  # run indexes to use
# Method to use
# -------------
# for 1d data
method_tups_1d = [(False, None, 200, 100),
                  (True, None, 1000, 100),
                  (True, 1, 1000, 100)]
# for 2d data
method_tups_2d = [(False, None, 400, 250),
                  (True, None, 2000, 250),
                  (True, 1, 2000, 250)]
# Set up
# ------
problem_tups = []
for fit_func in fit_func_list:
    for data_func in data_func_list:
        for data_type in data_type_list:
            problem_tups.append((fit_func, data_func, data_type))
method_tups = method_tups_1d

# Get results

In [ ]:
# Load data
# ---------
load_data = True
if load_data:
    # Load the NS runs
    results_dict = bsr.results_utils.load_data(
        problem_tups, method_tups, inds, base_dir='chains')
else:
    # Just get a dictionary of information on the likelihoods
    # Can be used to load cached results tables without loading all the NS runs
    results_dict = bsr.results_utils.make_base_dict(problem_tups, method_tups)

In [ ]:
# Get results tables
# ------------------
results_df = bsr.results_tables.get_results_df(
    results_dict, load=True, n_simulate=10)

In [ ]:
# # Check T posterior
# # -----------------
# results_df['$P(T=1)$'].xs('True_1_2000_250', level='method key')

In [ ]:
# Make LaTeX results table
# ------------------------
# Paper uses estimator = ['$y(0.5)$'] for 1d and['$y(0.5,0.5)$'] for 2d
estimator = ['$y(0.5)$']
calc_types = ['combined', 'bs gain']
# get df
df_temp = results_df.loc[pd.IndexSlice[:, :, calc_types, :], :][estimator].unstack('method key') #['$P(N=1)$'].unstack('method key')
# add number of samples
df_temp_nsamp = results_df.loc[pd.IndexSlice[:, :, ['combined'], ['value']], :]['nsample'].unstack('method key')
for pk in set(df_temp.index.get_level_values('problem key')):
    df_temp.loc[(pk, '\# samples', 'value'), :] = df_temp_nsamp.loc[(pk, 'combined', 'value'), :].values
# add true value
df_temp ['true value'] = np.nan
for pk in set(df_temp.index.get_level_values('problem key')):
    df_temp.loc[(pk, 'combined', 'value'), 'true value'] = bsr.data.get_true_value(
        bf.gg_1d, int(pk[-1]))
cols = ['vanilla', 'dynamic adaptive', 'adaptive', 'true value']
df_temp.columns = cols
df_temp.loc[pd.IndexSlice[:, ['bs gain'], :], ['vanilla']] = np.nan
df_temp = df_temp[[cols[i] for i in [0, 2, 1]]]
df_temp = df_temp.sort_index()
df_temp

In [ ]:
str_map = {'combined': r'$y(0.5;\bm{\theta})$',
           'bs gain': 'efficiency gain',
           'NaN': ''}
try:
    import texunc
    df_tex = texunc.print_latex_df(
        df_temp,
        min_dp=0, max_dp=4, max_power=5, str_map=str_map)
    df_tex
except ImportError:
    print("Install texunc for LaTeX formatting")

# Plot Results

In [ ]:
# Plot Odds
# ---------
odds_fig_list = bsr.paper_plots.odds(
    results_df, max_unc=True, nruns=len(inds)) # .xs('gg_1d_gg_1d_1', drop_level=False))

In [ ]:
# Plot multi
# ----------
multi_fig_list = bsr.paper_plots.multi(
    results_dict,
    prob_condition=lambda x: True,  # x == ('gg_2d', 'gg_2d', 1),
    meth_condition=lambda x: True)  # x[0] and x[1] == 1)

In [ ]:
# Plot split
# ----------
import bsr.plotting
plt.close('all')
split_fig_list = bsr.paper_plots.split(
    results_dict,
    prob_condition=lambda x: True,  # x == ('gg_2d', 'get_image', 1),  # ((x[0] == 'nn_1l' and x[1] == 'get_image') and x[1] or 'adfam' in x[0]) and x[2] == 1,
    meth_condition=lambda x: not x[0])

# Extra plots not used in paper

### Plot data

In [ ]:
for data_func in [bf.gg_1d]:  # [bsr.data.get_image, bf.gg_2d]:
    if data_func.__name__[-2:] == '1d':
        y_error_sigma = 0.075
        x_error_sigma = 0.075
        npoints = 100
    else:
        y_error_sigma = 0.3
        x_error_sigma = None
        npoints = 32
    for data_type in [1, 2, 3]:
        fig = bsr.plotting.plot_runs(
            [], [], data=bsr.data.generate_data(data_func, data_type, y_error_sigma, x_error_sigma=x_error_sigma, npoints=npoints),
            plot_data=True)

###  Prior Plot

In [ ]:
temp_likelihood = bsr.likelihoods.FittingLikelihood(
    bsr.data.generate_data(bf.gg_1d, 1, 0.1), bf.gg_1d, 10,  # NB number of funcs may affect prior plot
    adaptive=True)
prior_fig = bsr.plotting.plot_prior(temp_likelihood, 200)

### getdist plot

This requires ``getdist``.

In [ ]:
import getdist.plots
prob_key = ('gg_1d', 'gg_1d', 3)
meth_key = method_tups[0]
print(meth_key)
nfunc = 3
likelihood = results_dict[prob_key][meth_key]['likelihood_list'][0 if meth_key[0] else nfunc - 1]
run = results_dict[prob_key][meth_key]['run_list'][0 if meth_key[0] else nfunc - 1]

names = likelihood.get_param_names()
print(names)
latex_names = [name[1:-1] for name in likelihood.get_param_latex_names()]
print(latex_names)
logw = nestcheck.ns_run_utils.get_logw(run)
weights = np.exp(logw - logw.max())
# remove zero weights as they can throw errors
inds = np.nonzero(weights)
samples = getdist.MCSamples(
    samples=run['theta'][inds, :], weights=weights[inds],
    names=names,
    labels=latex_names,
    label='run')
g = getdist.plots.getSubplotPlotter()
g.triangle_plot(samples, names, shaded=True)